<a href="https://colab.research.google.com/github/Erickrus/llm/blob/main/AI_Paper_Highlighter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Paper Highlighter

In [2]:
#@title installations
!pip install -q PyPDF2 pdfplumber pymupdf
!apt-get -qq update -y
!apt-get install -y -qq poppler-utils
!pip3 -q install pdf2image
!pip3 install -q -U langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 342.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.3 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package poppler-utils.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up 

In [4]:
#@title setup GOOGLE_API_KEY
#@markdown Gemini Flash 2.5 is used to generate the highlight
import getpass
import os
GOOGLE_API_KEY = getpass.getpass("Enter your GOOGLE_API_KEY: ")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

Enter your GOOGLE_API_KEY: ··········


In [9]:
#@title highlight paper

import os
import requests


PAPER_NAME = "2507.02259" #@param {type:"string"}

# download pdf file directly from https://arxiv.org/html/2410.01943 and save the pdf
url = f'https://arxiv.org/pdf/{PAPER_NAME}'
r = requests.get(url, allow_redirects=True)
# the name will be changed, how to get the name from arxiv?
FILE_NAME = r.headers['Content-Disposition'].split('filename=')[1]
FILE_NAME= FILE_NAME.strip('"')
open(FILE_NAME, 'wb').write(r.content)
print(f"FILE_NAME: {FILE_NAME}")

# FILE_NAME = "2410.01943v1.pdf" #@param {type:"string"}

HTML_FILENAME = FILE_NAME[:-4] + ".html"
PDF_FILENAME = FILE_NAME[:-4] + ".pdf"
HTML_OUT_FILENAME = FILE_NAME[:-4] + "_out.html"
PDF_OUT_FILENAME = FILE_NAME[:-4] + "_out.pdf"


print(f"HTML_FILENAME: {HTML_FILENAME}")
print(f"PDF_FILENAME: {PDF_FILENAME}")
print(f"HTML_OUT_FILENAME: {HTML_OUT_FILENAME}")
print(f"PDF_OUT_FILENAME: {PDF_OUT_FILENAME}")





import requests
from bs4 import BeautifulSoup
url = f"https://arxiv.org/html/{PAPER_NAME}"
response = requests.get(url)
html_content = response.content.decode('utf-8')

# use beautiful soup to extract texts under <article> tag, however, will not include the section where class='ltx_bibliography'
# which means everything under and include this section will not be included in the extract texts.
soup = BeautifulSoup(html_content, 'html.parser')

# Find the article tag
article_tag = soup.find('article')

extracted_text = ""
if article_tag:
    # Remove the bibliography section if it exists within the article
    bibliography_section = article_tag.find(class_='ltx_bibliography')
    if bibliography_section:
        bibliography_section.extract()

    # Extract all text from the article tag
    extracted_text = article_tag.get_text(separator='\n')
    #print(extracted_text)
    print(f"extracted text from {url}")

print("invoke LLM to higlight paper text")
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

PROMPT = f'''This is the entire paper in text:
Now can you mark the highlight tag for this paper ? highlight means something important in the key paragraph , they are key sentence or key phrase or words. using <highlight>...</highlight> to wrap them up. Make sure highlight is not the entire paragraph.
https://arxiv.org/html/{PAPER_NAME}

```
{extracted_text}
```
'''
response = llm.invoke(PROMPT)

with open(HTML_FILENAME, "w") as f:
    content = response.content.replace("<highlight>", "&lt;highlight&gt;").replace("</highlight>", "&lt;/highlight&gt;").replace("\n", "<br/>")
    f.write(f"""<html><body><pre>{content}</pre></body></html>""")
    print(f"highlighted {HTML_FILENAME} is saved")


with open('/content/highlight.py', 'w') as f:
    f.write('''def convert_highlights(input_file, output_file):
    try:
        # Read the input HTML file
        with open(input_file, 'r', encoding='utf-8') as file:
            content = file.read()

        # Replace <highlight> tags with span tags containing yellow background style
        converted_content = content.replace(
            '&lt;highlight&gt;',
            '<span style="background-color: yellow;">'
        ).replace(
            '&lt;/highlight&gt;',
            '</span>'
        )

        # Write the converted content to output file
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(converted_content)

        print(f"Conversion complete! Output saved to {output_file}")

    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example usage
if __name__ == "__main__":
    import os,sys
    input_html = sys.argv[1]
    output_html = sys.argv[2]
    convert_highlights(input_html, output_html)
''')

with open("/content/highlight_pdf.py", "w") as f:
    f.write('''
import re
import fitz  # PyMuPDF
import sys
from typing import List
from bs4 import BeautifulSoup

def extract_highlighted_texts(html_file: str) -> List[str]:
    """Extract text between <highlight> and </highlight> markers, stripping nested HTML tags."""
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()
    # Use regex to find text between <highlight> and </highlight>
    pattern = r'&lt;highlight&gt;(.*?)&lt;/highlight&gt;'
    highlighted_texts = re.findall(pattern, html_content, re.DOTALL)
    # Strip nested HTML tags and clean up text
    cleaned_texts = []
    for text in highlighted_texts:
        # Parse with BeautifulSoup to remove HTML tags
        soup = BeautifulSoup(text, 'html.parser')
        cleaned_text = ' '.join(soup.get_text().split())  # Normalize whitespace
        if cleaned_text:
            cleaned_texts.append(cleaned_text)
    return cleaned_texts

def add_highlight_to_pdf(pdf_file: str, output_file: str, highlighted_texts: List[str]):
    """Add highlight annotations to the PDF for each highlighted text."""
    doc = fitz.open(pdf_file)
    highlights_added = False

    for target_text in highlighted_texts:
        found = False
        # Search each page for the target text
        for page_num in range(len(doc)):
            page = doc[page_num]
            # Search for the exact text (case-insensitive)
            rects = page.search_for(target_text, flags=fitz.TEXTFLAGS_SEARCH )
            if rects:
                # Highlight all found instances on this page
                for rect in rects:
                    annot = page.add_highlight_annot(rect)
                    if annot:
                        annot.update()
                        # print(f"Highlighted text: '{target_text}' on page {page_num + 1} at coordinates "
                        #       f"({rect.x0:.2f}, {rect.y0:.2f}, {rect.x1:.2f}, {rect.y1:.2f})")
                        highlights_added = True
                found = True
        if not found:
            print(f"Warning: Could not find match for '{target_text}' in PDF. Skipping.")

    if highlights_added:
        doc.save(output_file, garbage=4, deflate=True)
        print(f"PDF with highlights is saved to {output_file}")
    else:
        print("No highlights were added to the PDF.")

    doc.close()

def main(html_file: str, pdf_file: str, output_file: str):
    # Step 1: Extract highlighted texts from HTML
    highlighted_texts = extract_highlighted_texts(html_file)
    if not highlighted_texts:
        print("No highlighted texts found in HTML.")
        return

    # Step 2: Add highlights to PDF
    add_highlight_to_pdf(pdf_file, output_file, highlighted_texts)

if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Usage: python3 highlight_pdf.py input.html input.pdf output.pdf")
        sys.exit(1)

    html_file, pdf_file, output_file = sys.argv[1], sys.argv[2], sys.argv[3]
    main(html_file, pdf_file, output_file)
''')

!python3 highlight.py {HTML_FILENAME} {HTML_OUT_FILENAME}
!python3 highlight_pdf.py {HTML_FILENAME} {PDF_FILENAME} {PDF_OUT_FILENAME}

FILE_NAME: 2507.02259v1.pdf
HTML_FILENAME: 2507.02259v1.html
PDF_FILENAME: 2507.02259v1.pdf
HTML_OUT_FILENAME: 2507.02259v1_out.html
PDF_OUT_FILENAME: 2507.02259v1_out.pdf
extracted text from https://arxiv.org/html/2507.02259
invoke LLM to higlight paper text
highlighted 2507.02259v1.html is saved
Conversion complete! Output saved to 2507.02259v1_out.html
PDF with highlights is saved to 2507.02259v1_out.pdf


In [ ]:
#@title download
from google.colab import files
files.download(PDF_OUT_FILENAME)

In [ ]:
#@title display paper
# from IPython.core.display import display, HTML
# with open(HTML_OUT_FILENAME, "r") as f:
#     display(HTML(f.read()))

import requests
import io
import base64
from pdf2image import convert_from_bytes
from IPython.display import HTML, display

def display_scrollable_pdf(filename, width=800, height=450):

    try:
        with open(filename, "rb") as f:
            pdf_bytes = f.read()
        images = convert_from_bytes(pdf_bytes)

        img_tags = []
        for img in images:
            buffer = io.BytesIO()
            img.save(buffer, format='PNG')
            img_base64 = base64.b64encode(buffer.getvalue()).decode()
            img_tag = f'<img src="data:image/png;base64,{img_base64}" style="width:100%; margin-bottom:10px;" />'
            img_tags.append(img_tag)

        html_code = f"""
        <div style="width:{width}px; height:{height}px; overflow:auto; border:1px solid #ccc;
                    padding:10px; border-radius:8px;">
            {''.join(img_tags)}
        </div>
        """
        display(HTML(html_code))

    except Exception as e:
        display(HTML(f"<div style='color:red;'>Error loading PDF: {e}</div>"))

display_scrollable_pdf(
    f"/content/{PDF_OUT_FILENAME}",
    width=800,
    height=450
)